In [1]:
import pandas as pd
import os
from openpyxl import load_workbook

In [2]:
# imorting the student profiles data
student_profiles = pd.read_excel('student profiles.xlsx')
student_profiles.dtypes


Student ID     int64
Birthday      object
dtype: object

In [3]:
student_profiles.shape

(91, 2)

In [4]:
student_profiles.isna().sum()

Student ID     0
Birthday      10
dtype: int64

The birthday dates for 10 students were not provided, this needs to be dropped as it will be of no use.

In [5]:
# dropping rows 
student_profiles.dropna(how='any',axis=0,inplace=True)

student_profiles.isna().sum()

Student ID    0
Birthday      0
dtype: int64

In [6]:
student_profiles

,Student ID,Birthday
0,1,2012-09-03 00:00:00
1,2,2011-09-07 00:00:00
2,3,2011-04-08 00:00:00
3,4,2013-07-07 00:00:00
4,5,2012-04-05 00:00:00
...,...,...
76,77,2012-12-19 00:00:00
82,83,2011
88,89,2013
89,90,2013


the birthday column is not consistent as some of the birthday does not have the birth month and day which would be needed when calculating the age in years and month, so we may need to exclude this.

In [7]:
student_profiles=student_profiles[0:77]
student_profiles

,Student ID,Birthday
0,1,2012-09-03 00:00:00
1,2,2011-09-07 00:00:00
2,3,2011-04-08 00:00:00
3,4,2013-07-07 00:00:00
4,5,2012-04-05 00:00:00
...,...,...
72,73,2010-05-20 00:00:00
73,74,2012-11-09 00:00:00
74,75,2013-03-09 00:00:00
75,76,2012-11-14 00:00:00


In [8]:
# converting the Birthday column to datetime
student_profiles.Birthday=pd.to_datetime(student_profiles.Birthday).dt.date
student_profiles.rename(columns={'Student ID':'Student_ID'},inplace=True)
student_profiles

C:\Users\HP\AppData\Local\Temp\ipykernel_4116\2980876507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_profiles.Birthday=pd.to_datetime(student_profiles.Birthday).dt.date
C:\Users\HP\AppData\Local\Temp\ipykernel_4116\2980876507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_profiles.rename(columns={'Student ID':'Student_ID'},inplace=True)


,Student_ID,Birthday
0,1,2012-09-03
1,2,2011-09-07
2,3,2011-04-08
3,4,2013-07-07
4,5,2012-04-05
...,...,...
72,73,2010-05-20
73,74,2012-11-09
74,75,2013-03-09
75,76,2012-11-14


Organizing and transforming the Mock Exam data

The Mock Exam data is currently in a wide format. It's will be good to use the `melt()` function from the pandas library to transform it into a cleaner long format. After this, the data will be merged with the student profiles data. Additionally, age columns will be generated for each student, calculated based on birthdate and mock exam completion date. Similarly, a difficulty index column will be added. These actions will be performed for each worksheet and subsequently saved as individual files to a designated folder.

In [59]:
workbook=load_workbook('Mock Exam Data (Labelled).xlsx')
sheet_names=workbook.sheetnames
count=0
for sheet in sheet_names:
    sheets=workbook[sheet]
    cell_value=sheets['A1'].value
    df=pd.read_excel('Mock Exam Data (Labelled).xlsx',sheet_name=sheet)
    header_names = df.columns.tolist()

    # Get the values from the first row
    first_row_values = df.iloc[0].tolist()

    # Concatenate header names with corresponding values
    concatenated_values = [f"{header}: {value}" for header, value in zip(header_names, first_row_values)]

    # Add a new row with concatenated values to the DataFrame
    concatenated_row = pd.DataFrame([concatenated_values], columns=header_names)
    result_df = pd.concat([concatenated_row, df[1:]], ignore_index=True)

    new_header = result_df.iloc[0]
    result_df = result_df[1:]
    result_df.columns = new_header
    result_df.rename(columns={f'{cell_value}: Question Type':'question_type','Unnamed: 1: Question':'question_number','Unnamed: 2: Answer':'correct_answer'}, inplace=True)
    result_df.drop(columns='Timestamp: Full Name:',inplace=True)
    
    pivoted_data=result_df.melt(id_vars=['question_type','question_number','correct_answer'],var_name='Student_ID',value_name='Response')


    # code to create a date commpleted column  it from the student id
    pivoted_data['Date_Completed']=pivoted_data['Student_ID'].str.extract(r'(\d{4}-\d{2}-\d{2})')

    pivoted_data['Student_ID']=pivoted_data['Student_ID'].str.extract(r': (\d+)$')
    pivoted_data['Student_ID']=pivoted_data['Student_ID'].astype('int64')

    # create a column 'is_right' to check if response is correct categorized as 1 or 0 
    pivoted_data['is_right']=pivoted_data.apply(lambda x: 1 if x['Response']==x['correct_answer'] else 0,axis=1)
    # code to calculate the difficult index for each question (number of corrct responses over total number of responses)
    difficulty_index = pivoted_data.groupby('question_number')['is_right'].mean()

    # Add the Difficulty Index as a new column to the DataFrame
    pivoted_data['difficulty_index'] = pivoted_data['question_number'].map(round(difficulty_index,2))

    
    pivoted_data=pd.merge(pivoted_data,student_profiles,on='Student_ID',how='inner')

    # code to calculate age in years and month from date of birth and date completed 
    pivoted_data['Age_years']=((pd.to_datetime(pivoted_data['Date_Completed'])-pd.to_datetime(pivoted_data['Birthday']))/365).dt.days
    pivoted_data['Age_months'] = ((pd.to_datetime(pivoted_data['Date_Completed']) - pd.to_datetime(pivoted_data['Birthday'])).dt.days % 365) // 30
    pivoted_data['new_id']=pivoted_data['Student_ID'].astype('str')+'|'+pivoted_data['Date_Completed'].astype('str')
    pivoted_data['genre']=cell_value
    pivoted_data['title']=sheet

    df1=pivoted_data.groupby('new_id')['is_right'].agg(['mean','sum']).reset_index()
    df1.rename(columns={'mean':'percentage_correct','sum':'total_correct'},inplace=True)
    df1=df1.round(4)
    pivoted_data['Age_years'] = pivoted_data['Age_years'].astype('int64')
    pivoted_data['Age_months'] = pivoted_data['Age_months'].astype('int64')
    pivoted_data=pd.merge(pivoted_data,df1,on='new_id',how='inner')
    pivoted_data['total_correct']=pivoted_data['total_correct'].astype('str')+'/'+str(len(pivoted_data['question_number'].unique()))
    pivoted_data['Date_Completed_year']= pd.to_datetime(pivoted_data['Date_Completed']).dt.year
    count+=pivoted_data.shape[0]
    pivoted_data.to_excel(f'~/Downloads/data cleaning/Mock cleaned/{sheet}.xlsx',index=False)
    print(f'loaded passage {sheet}: {pivoted_data.shape[0]} rows')
    
print(f'loaded a total of {count} rows')


loaded passage bluebird: 800 rows
loaded passage dorian: 616 rows
loaded passage jane eyre: 609 rows
loaded passage notre dame: 450 rows
loaded passage metamorphosis: 875 rows
loaded passage sleepy hollow: 560 rows
loaded passage the hobbit: 90 rows
loaded passage those winter: 306 rows
loaded a total of 4306 rows


<h5>Combine all the individual files in the folder <h5>

In [60]:
files=os.listdir(os.path.expanduser('~/Downloads/data cleaning/Mock cleaned').replace('\\','/'))
files
    

['bluebird.xlsx',
 'dorian.xlsx',
 'jane eyre.xlsx',
 'metamorphosis.xlsx',
 'notre dame.xlsx',
 'sleepy hollow.xlsx',
 'the hobbit.xlsx',
 'those winter.xlsx']

In [61]:
combined_df=pd.DataFrame()
for file in files:
    passage=pd.read_excel(f'~/Downloads/data cleaning/Mock cleaned/{file}')
    combined_df=pd.concat([combined_df,passage],ignore_index=True)

combined_df.head()

,question_type,question_number,correct_answer,Student_ID,Response,Date_Completed,is_right,difficulty_index,Birthday,Age_years,Age_months,new_id,genre,title,percentage_correct,total_correct,Date_Completed_year
0,H,1,D,61,C,2020-08-30,0,0.31,2010-05-05,10,4,61|2020-08-30,poem,bluebird,0.72,18/25,2020
1,M,2,D,61,D,2020-08-30,1,0.56,2010-05-05,10,4,61|2020-08-30,poem,bluebird,0.72,18/25,2020
2,G,3,B,61,B,2020-08-30,1,0.78,2010-05-05,10,4,61|2020-08-30,poem,bluebird,0.72,18/25,2020
3,M,4,D,61,C,2020-08-30,0,0.19,2010-05-05,10,4,61|2020-08-30,poem,bluebird,0.72,18/25,2020
4,M,5,B,61,B,2020-08-30,1,0.72,2010-05-05,10,4,61|2020-08-30,poem,bluebird,0.72,18/25,2020


In [62]:
combined_df.shape

(4306, 17)

In [63]:
combined_df.isna().sum()

question_type           0
question_number         0
correct_answer          0
Student_ID              0
Response               67
Date_Completed          0
is_right                0
difficulty_index        0
Birthday                0
Age_years               0
Age_months              0
new_id                  0
genre                   0
title                   0
percentage_correct      0
total_correct           0
Date_Completed_year     0
dtype: int64

In [64]:

combined_df.to_excel('Mock exam data combined.xlsx',index=False)

<h4> Worksheet data

In [55]:
workbook=load_workbook('worksheet1.xlsx')
sheet_names=workbook.sheetnames
sheet_names.remove( 'robin -considered')
sheet_names.remove('robin -happened')
count=0
for sheet in sheet_names:
    sheets=workbook[sheet]
    cell_value=sheets['A1'].value
    df=pd.read_excel('worksheet1.xlsx',sheet_name=sheet)
    header_names = df.columns.tolist()
    # Get the values from the first row
    first_row_values = df.iloc[0].tolist()

    # Concatenate header names with corresponding values
    concatenated_values = [f"{header}: {value}" for header, value in zip(header_names, first_row_values)]

    # Add a new row with concatenated values to the DataFrame
    concatenated_row = pd.DataFrame([concatenated_values], columns=header_names)
    result_df = pd.concat([concatenated_row, df[1:]], ignore_index=True)

    new_header = result_df.iloc[0]
    result_df = result_df[1:]
    result_df.columns = new_header
    
    result_df.rename(columns={f'{cell_value}: Question Type':'question_type','Unnamed: 1: Question':'question_number','Unnamed: 2: Answer':'correct_answer'}, inplace=True)
    result_df.drop(columns='Date of Completion: Name',inplace=True)
    
    pivoted_data=result_df.melt(id_vars=['question_type','question_number','correct_answer'],var_name='Student_ID',value_name='Response')


    # code to create a date commpleted column  it from the student id
    pivoted_data['Date_Completed']=pivoted_data['Student_ID'].str.extract(r'(\d{4}-\d{2}-\d{2})')

    pivoted_data['Student_ID']=pivoted_data['Student_ID'].str.extract(r': (\d+)$')
    pivoted_data=pivoted_data.dropna(subset=['Student_ID'])
    pivoted_data['Student_ID']=pivoted_data['Student_ID'].astype('int64')

    # create a column 'is_right' to check if response is correct categorized as 1 or 0 
    pivoted_data['is_right']=pivoted_data.apply(lambda x: 1 if x['Response']==x['correct_answer'] else 0,axis=1)
    # code to calculate the difficult index for each question (number of corrct responses over total number of responses)
    difficulty_index = pivoted_data.groupby('question_number')['is_right'].mean()

    # Add the Difficulty Index as a new column to the DataFrame
    pivoted_data['difficulty_index'] = pivoted_data['question_number'].map(round(difficulty_index,2))

    
    pivoted_data=pd.merge(pivoted_data,student_profiles,on='Student_ID',how='inner')

    # code to calculate age in years and month from date of birth and date completed 
    pivoted_data['Age_years']=((pd.to_datetime(pivoted_data['Date_Completed'])-pd.to_datetime(pivoted_data['Birthday']))/365).dt.days
    pivoted_data['Age_months'] = ((pd.to_datetime(pivoted_data['Date_Completed']) - pd.to_datetime(pivoted_data['Birthday'])).dt.days % 365) // 30
    pivoted_data['new_id']=pivoted_data['Student_ID'].astype('str')+'|'+pivoted_data['Date_Completed'].astype('str')
    pivoted_data['genre']=cell_value
    pivoted_data['title']=sheet
    df1=pivoted_data.groupby('new_id')['is_right'].agg(['mean','sum']).reset_index()
    df1.rename(columns={'mean':'percentage_correct','sum':'total_correct'},inplace=True)
    pivoted_data['Age_years'] = pivoted_data['Age_years'].astype('int64')
    pivoted_data['Age_months'] = pivoted_data['Age_months'].astype('int64')
    pivoted_data=pd.merge(pivoted_data,df1,on='new_id',how='inner')
    pivoted_data['total_correct']=pivoted_data['total_correct'].astype('str')+'/'+str(len(pivoted_data['question_number'].unique()))
    pivoted_data['Date_Completed_year']= pd.to_datetime(pivoted_data['Date_Completed']).dt.year
    count+=pivoted_data.shape[0]    
    pivoted_data.to_excel(f'~/Downloads/data cleaning/Work cleaned/{sheet}.xlsx',index=False)
    print(f'loaded passage {sheet}: {pivoted_data.shape[0]} rows')
    

print(f'loaded a total of {count} rows')

loaded passage anne (2): 96 rows
loaded passage anne (3): 150 rows
loaded passage 1984: 117 rows
loaded passage alice: 198 rows
loaded passage baskervilles: 288 rows
loaded passage black beauty: 36 rows
loaded passage brazilian: 120 rows
loaded passage charlottes: 40 rows
loaded passage court life: 104 rows
loaded passage dorian: 100 rows
loaded passage dreams of a giant life: 36 rows
loaded passage economics (3): 40 rows
loaded passage economics (2): 50 rows
loaded passage fauntleroy: 168 rows
loaded passage frankenstein: 84 rows
loaded passage friendship: 126 rows
loaded passage frost (4): 45 rows
loaded passage frost (3): 90 rows
loaded passage frost (2): 60 rows
loaded passage gatsby: 108 rows
loaded passage gibson: 17 rows
loaded passage growth (3): 13 rows
loaded passage growth (2): 26 rows
loaded passage gulliver: 30 rows
loaded passage homing pigeon (3): 30 rows
loaded passage homing pigeon (2): 36 rows
loaded passage honeybees: 240 rows
loaded passage huck: 120 rows
loaded pas

In [56]:
files2=os.listdir(os.path.expanduser('~/Downloads/data cleaning/Work cleaned').replace('\\','/'))
files2

['1984.xlsx',
 'alice.xlsx',
 'anne (2).xlsx',
 'anne (3).xlsx',
 'baskervilles.xlsx',
 'black beauty.xlsx',
 'brazilian.xlsx',
 'charlottes.xlsx',
 'court life.xlsx',
 'dorian.xlsx',
 'dreams of a giant life.xlsx',
 'economics (2).xlsx',
 'economics (3).xlsx',
 'fauntleroy.xlsx',
 'frankenstein.xlsx',
 'friendship.xlsx',
 'frost (2).xlsx',
 'frost (3).xlsx',
 'frost (4).xlsx',
 'gatsby.xlsx',
 'gibson.xlsx',
 'growth (2).xlsx',
 'growth (3).xlsx',
 'gulliver.xlsx',
 'homing pigeon (2).xlsx',
 'homing pigeon (3).xlsx',
 'honeybees.xlsx',
 'huck.xlsx',
 'i have a dream.xlsx',
 'jabberwocky.xlsx',
 'jane eyre.xlsx',
 'jekyll (2).xlsx',
 'jekyll (3).xlsx',
 'jekyll (4).xlsx',
 'lion.xlsx',
 'little women.xlsx',
 'lochinvar (2).xlsx',
 'lochinvar (3).xlsx',
 'lost treasure.xlsx',
 'magic city.xlsx',
 'marathon.xlsx',
 'matilda.xlsx',
 'meg.xlsx',
 'no enemies (2).xlsx',
 'no enemies (3).xlsx',
 'nothing gold (2).xlsx',
 'nothing gold (3).xlsx',
 'notre dame.xlsx',
 'oliver twist master.xls

In [57]:
combined_df2=pd.DataFrame()
for file in files2:
    passage=pd.read_excel(f'~/Downloads/data cleaning/Work cleaned/{file}')
    combined_df2=pd.concat([combined_df2,passage],ignore_index=True)

combined_df2

,question_type,question_number,correct_answer,Student_ID,Response,Date_Completed,is_right,difficulty_index,Birthday,Age_years,Age_months,new_id,genre,title,percentage_correct,total_correct,Date_Completed_year
0,M,1,A,15,A,2022-03-14,1,1.00,2012-08-26,9,6,15|2022-03-14,fiction,1984,0.615385,8/13,2022
1,M,2,B,15,B,2022-03-14,1,0.89,2012-08-26,9,6,15|2022-03-14,fiction,1984,0.615385,8/13,2022
2,"M, T/F",3,C,15,C,2022-03-14,1,0.89,2012-08-26,9,6,15|2022-03-14,fiction,1984,0.615385,8/13,2022
3,M,4,A,15,A,2022-03-14,1,1.00,2012-08-26,9,6,15|2022-03-14,fiction,1984,0.615385,8/13,2022
4,M,5,B,15,B,2022-03-14,1,0.78,2012-08-26,9,6,15|2022-03-14,fiction,1984,0.615385,8/13,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6572,NaN,9,C,21,D,2022-04-27,0,0.67,2013-03-30,9,1,21|2022-04-27,fiction,wizard,0.692308,9/13,2022
6573,NaN,10,C,21,C,2022-04-27,1,0.50,2013-03-30,9,1,21|2022-04-27,fiction,wizard,0.692308,9/13,2022
6574,NaN,11,A,21,A,2022-04-27,1,0.83,2013-03-30,9,1,21|2022-04-27,fiction,wizard,0.692308,9/13,2022
6575,NaN,12,B,21,B,2022-04-27,1,0.33,2013-03-30,9,1,21|2022-04-27,fiction,wizard,0.692308,9/13,2022


In [58]:
combined_df2.to_excel('worksheet_combined.xlsx',index=False)